### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
import pathlib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import tkinter as tk
from tkinter import filedialog

Connecting google colab to google drive where the dataset is.

This is the link for dataset https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset

We have used only training images from this.

There are 10k images of which 5k are with mask and 5k without mask.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir="/content/drive/MyDrive/FaceMaskDataset"
data_dir=pathlib.Path(data_dir)

Declaring batch size.

Since images used are of different sizes but for training model we need images of same size therefore we have to resize all the images of dataset. 

In [4]:
batch_size=8
img_height=64
img_width=64

Splitting dataset into 2 parts for training the model and testing it.

We are dividing dataset in 80:20 ratio.



Seed is used for shuffling data.

We use seed value while creating training and test dataset.

Giving a seed value makes sure every time same we get same train and test dataset for changing hyperparameters.

In [5]:
ds_train=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,validation_split=0.2,subset='validation',seed=80,
    image_size=(img_height,img_width), batch_size=batch_size
)


ds_val=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,validation_split=0.2,subset='validation',seed=80,
    image_size=(img_height,img_width), batch_size=batch_size
)

Found 10000 files belonging to 2 classes.
Using 2000 files for validation.
Found 10000 files belonging to 2 classes.
Using 2000 files for validation.


In [6]:
class_names=ds_train.class_names
print(class_names)

['WithMask', 'WithoutMask']


For memory optimization and faster execution.

In [7]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
ds_train=ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
ds_val=ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
n_class=2

### Building CNN Network



First we rescaled all the values to 0-1.

Rescaling increses the accuracy of model.

MODEL

input--->Conv2D+ReLU--->MaxPool--->Conv2D+ReLU--->MaxPool--->Conv2D+ReLU--->MaxPool--->Dropout--->Flatten--->Fully Connected--->Fully Connected

We used dropout layer to avoid overfitting

In [30]:
model =Sequential([

  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),

  layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),

  layers.MaxPooling2D(),

  layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),

  layers.MaxPooling2D(),

  layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),

  layers.MaxPooling2D(),

  layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),

  layers.MaxPooling2D(),

  layers.Dropout(0.2),

  layers.Flatten(),

  layers.Dense(units=64, activation='relu'),

  layers.Dense(n_class)

  ])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_3 (Rescaling)      (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 6, 6, 64)         

Compling model using adam optimizer

In [34]:
model.compile(optimizer ="adam", loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics =['accuracy'])

mymodel=model.fit(ds_train, validation_data=ds_val, epochs=9)

Epoch 1/9
250/250 [==============================] - 13s 48ms/step - loss: 0.0242 - accuracy: 0.9920 - val_loss: 0.0175 - val_accuracy: 0.9925
Epoch 2/9
250/250 [==============================] - 12s 46ms/step - loss: 0.0109 - accuracy: 0.9965 - val_loss: 0.0050 - val_accuracy: 0.9980
Epoch 3/9
250/250 [==============================] - 11s 46ms/step - loss: 0.0174 - accuracy: 0.9950 - val_loss: 0.0148 - val_accuracy: 0.9935
Epoch 4/9
250/250 [==============================] - 12s 47ms/step - loss: 0.0283 - accuracy: 0.9895 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 5/9
250/250 [==============================] - 12s 47ms/step - loss: 0.0129 - accuracy: 0.9945 - val_loss: 0.0077 - val_accuracy: 0.9980
Epoch 6/9
250/250 [==============================] - 12s 47ms/step - loss: 0.0188 - accuracy: 0.9930 - val_loss: 0.0071 - val_accuracy: 0.9965
Epoch 7/9
250/250 [==============================] - 12s 47ms/step - loss: 0.0113 - accuracy: 0.9965 - val_loss: 9.2556e-04 - val_accuracy: 1.

Achieved 99.95% accuracy on new dataset